<a href="https://colab.research.google.com/github/cnhzgb/MachineL/blob/main/RESNET_TINY_CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import ipdb
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from einops import rearrange
from torchsummary import summary
import time

device = "cuda" if torch.cuda.is_available() else "cpu"
print(torch.backends.mps.is_available())
print(torch.backends.mps.is_built())
device = torch.device("mps")
#print(device)

True
True


In [11]:
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]) # transforms.RandomHorizontalFlip(), transforms.RandomGrayscale()
dataset = datasets.CIFAR10(root="/Users/bin.guanb/code/MachineL/dataset/", transform=trans, download=False, train=True) # 5W张图片, 10种分类
loader = DataLoader(dataset, batch_size=100, shuffle=True)
batch_num,(image, label) = next(enumerate(loader))
print(len(dataset.classes), len(dataset), image.shape, label.shape) # 10; 5W; 100,3,32,32; 100

trans_test = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
dataset_test = datasets.CIFAR10(root="/Users/bin.guanb/code/MachineL/dataset/", transform=trans_test, download=False, train=False) # 1W张图片
loader_test = DataLoader(dataset_test, batch_size=100, shuffle=True)
criterion_test = nn.CrossEntropyLoss()

10 50000 torch.Size([100, 3, 32, 32]) torch.Size([100])


In [28]:
# https://www.cnblogs.com/emanlee/p/17138634.html

class Block(nn.Module):
  def __init__(self, inc, n_chans):
    super(Block, self).__init__()
    self.conv1 = nn.Conv2d(inc, n_chans, kernel_size=3, padding=1)
    self.conv2 = nn.Conv2d(n_chans, n_chans, kernel_size=3, padding=1)
    self.batch_norm = nn.BatchNorm2d(num_features=n_chans)

  def forward(self, x):
    x = self.conv1(x)
    x = F.relu(self.batch_norm(x))
    x = self.conv2(x)
    x = self.batch_norm(x)
    return x

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.pool = nn.MaxPool2d(2, 2)
        
        self.conv10 = nn.Conv2d(3,32,1,padding=0)
        self.conv11 = Block(3,32)

        self.conv20 = nn.Conv2d(32,64,1,padding=0)
        self.conv21 = Block(32,64)

        self.conv30 = nn.Conv2d(64,128,1,padding=0)
        self.conv31 = Block(64,128)

        self.conv40 = nn.Conv2d(128,256,1,padding=0)
        self.conv41 = Block(128,256)

        self.conv50 = nn.Conv2d(256,512,1,padding=0)
        self.conv51 = Block(256,512)
        
        self.fc1 = nn.Linear(512*2*2, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self,x):
        out1 = self.conv10(x) # 32,32,32
        out2 = self.conv11(x)
        x = self.pool(F.relu(out1 + out2)) # ,32,16,16

        out1 = self.conv20(x) # 64,16,16
        out2 = self.conv21(x)
        x = self.pool(F.relu(out1 + out2)) # ,64,8,8

        out1 = self.conv30(x) # 128,8,8
        out2 = self.conv31(x)
        x = self.pool(F.relu(out1 + out2)) # 128,4,4

        out1 = self.conv40(x) # 256,4,4
        out2 = self.conv41(x)
        x = self.pool(F.relu(out1 + out2)) # 256,2,2

        out1 = self.conv50(x) # 512,2,2
        out2 = self.conv51(x)
        x = F.relu(out1 + out2)
        
        x = x.view(-1,512*2*2)
        
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)

#print(model)
#summary(model, (3,32,32))

In [29]:
for epoch in range(1,10):
  total_loss = []
  errorTotal = 0
  startTime = time.time()
  for batch_idx, (img, label) in enumerate(loader):
    img = img.to(device)
    label = label.to(device)
    outputs = model(img)
    loss = criterion(outputs, label)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_loss.append(loss.item())
    maxV,maxIdx = outputs.max(dim=1)
    errorNum = torch.sum(torch.ne(maxIdx, label)).item()
    errorTotal += errorNum

    if(batch_idx % 100 == 99):
      print("epoch:{} batch:{} loss:{:.2f} mean:{:.2f} error:{}/100 errorTotal:{}/{} {:.2f}% time:{:.1f}".format(
          epoch, batch_idx, loss, np.mean(total_loss), errorNum, errorTotal, (batch_idx+1)*100, errorTotal/(batch_idx+1),
          time.time()-startTime))

epoch:1 batch:99 loss:1.48 mean:1.62 error:50/100 errorTotal:6004/10000 60.04% time:10.6
epoch:1 batch:199 loss:1.29 mean:1.47 error:46/100 errorTotal:10837/20000 54.19% time:15.2
epoch:1 batch:299 loss:1.09 mean:1.38 error:40/100 errorTotal:15188/30000 50.63% time:19.9
epoch:1 batch:399 loss:1.09 mean:1.32 error:44/100 errorTotal:19233/40000 48.08% time:24.5
epoch:1 batch:499 loss:1.00 mean:1.26 error:37/100 errorTotal:22996/50000 45.99% time:29.1
epoch:2 batch:99 loss:0.98 mean:0.91 error:43/100 errorTotal:3286/10000 32.86% time:4.6
epoch:2 batch:199 loss:0.89 mean:0.89 error:32/100 errorTotal:6352/20000 31.76% time:9.3
epoch:2 batch:299 loss:0.71 mean:0.88 error:25/100 errorTotal:9385/30000 31.28% time:13.9
epoch:2 batch:399 loss:0.62 mean:0.86 error:18/100 errorTotal:12210/40000 30.52% time:18.5
epoch:2 batch:499 loss:0.76 mean:0.85 error:29/100 errorTotal:15130/50000 30.26% time:23.1
epoch:3 batch:99 loss:0.51 mean:0.61 error:20/100 errorTotal:2120/10000 21.20% time:4.6
epoch:3 ba